In [ ]:
#%matplotlib widget

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%pip install lab_black
#%load_ext lab_black
%cd ..

In [ ]:
from pathlib import Path

import torch as th
import torch.nn.functional as F
import numpy as np
import yaml
from easydict import EasyDict

from src.utils import instantiate_from_config, get_device
from src.utils.vis import save_sdf_as_mesh, plot_sdfs

from tqdm import tqdm

In [ ]:
th.set_grad_enabled(False)
device = get_device()
#device="cpu"
device

# Load Pretrained Models

In [ ]:
gen32_args_path = "config/gen32/chair.yaml"
gen32_ckpt_path = "results/gen32/chair.pth"
sr64_args_path = "config/sr32_64/chair.yaml"
sr64_ckpt_path = "results/sr32_64/chair.pth"

In [ ]:
with open(gen32_args_path) as f:
    args1 = EasyDict(yaml.safe_load(f))
with open(sr64_args_path) as f:
    args2 = EasyDict(yaml.safe_load(f))

In [ ]:
model1 = instantiate_from_config(args1.model)
ckpt = th.load(gen32_ckpt_path, map_location=device)
model1.load_state_dict(ckpt["model_ema"])
model1 = model1.to(device)
model1.eval()
model1.training

In [ ]:
model2 = instantiate_from_config(args2.model)
ckpt = th.load(sr64_ckpt_path, map_location=device)
model2.load_state_dict(ckpt["model"])
model2 = model2.to(device)
model2.eval()
model2.training

In [ ]:
from src.models.gaussian_diffusion import create_gaussian_diffusion
noise_level_func = "sqrt_alphas_cumprod_next"
ddpm_sampler1 = create_gaussian_diffusion(noise_level_func=noise_level_func, **args1.ddpm_new.valid.params)
ddpm_sampler2 = create_gaussian_diffusion(noise_level_func=noise_level_func, **args2.ddpm_new.valid.params)

#ddpm_sampler1, ddpm_sampler2 = ddpm_sampler1.to(device), ddpm_sampler2.to(device)

In [ ]:
preprocessor1 = instantiate_from_config(args1.preprocessor, device=device)
preprocessor2 = instantiate_from_config(args2.preprocessor, device=device)

# Generate Low-Resolution ($32^3$)

Generates 5 low-resolution samples

In [ ]:
from src.utils.utils import seed_everything
seed_everything(69)

In [ ]:
out1 = ddpm_sampler1.ddim_sample_loop(model1, shape=(2, 1, 32, 32, 32), progress=True, device=device)
out1.shape

In [ ]:
out1 = preprocessor1.destandardize(out1)
print(out1.mean(), out1.min(), out1.max(), out1.var())
out1.shape

In [ ]:
out1_std = preprocessor1.standardize(out1)
print(out1_std.mean(), out1_std.min(), out1_std.max(), out1_std.var())
out1_std.shape

In [ ]:
from src.utils.vis import plot_sdfs
view_kwargs = {"azim": 30, "elev": 30, "roll": 0, "vertical_axis": "y"}
plot_sdfs(list(out1), view_kwargs=view_kwargs)

In [ ]:
from src.models.gaussian_diffusion import create_gaussian_diffusion
noise_level_func = "sqrt_alphas_cumprod_next"
ddpm_sampler1 = create_gaussian_diffusion(noise_level_func=noise_level_func, sigma_small=True, **args1.ddpm_new.valid.params)
ddpm_sampler2 = create_gaussian_diffusion(noise_level_func=noise_level_func, sigma_small=True, **args2.ddpm_new.valid.params)

from src.utils.vis import plot_sdfs
def plot_debug_func(x, iter, every_t=20, **plot_sdfs_kwargs):
    if iter % every_t == 0 or iter == len(ddpm_sampler1.use_timesteps)-1:
        title = f"iter: {iter} / {len(ddpm_sampler1.use_timesteps)}"
        plot_sdfs(x, title=title, **plot_sdfs_kwargs)

out1 = ddpm_sampler1.ddim_sample_loop(model1, shape=(3, 1, 32, 32, 32), progress=True, plot_debug_func=plot_debug_func, device=device)
out1.shape

In [ ]:
out1 = preprocessor1.destandardize(out1)
print(out1.mean(), out1.min(), out1.max(), out1.var())
#out1.shape
out1_std = preprocessor1.standardize(out1)
print(out1_std.mean(), out1_std.min(), out1_std.max(), out1_std.var())
#out1_std.shape
from src.utils.vis import plot_sdfs
view_kwargs = {"azim": 30, "elev": 30, "roll": 0, "vertical_axis": "y"}
plot_sdfs(list(out1), view_kwargs=view_kwargs)

In [ ]:
# save as an obj file
# for i, out in enumerate(out1):
#     save_sdf_as_mesh(f"gen32_{i}.obj", out, safe=True)

In [ ]:
lr_cond = F.interpolate(out1, (64, 64, 64), mode="nearest")
lr_cond = preprocessor2.standardize(lr_cond, 0)
out2 = ddpm_sampler1.ddim_sample_loop(lambda x, t: model2(th.cat([lr_cond, x], 1), t), shape=(out1.shape[0], 1, 64, 64, 64), progress=True, device=device)

out2 = preprocessor2.destandardize(out2, 1)

#for i, out in enumerate(out2):
#    save_sdf_as_mesh(f"sr64_{i}.obj", out, safe=True)

plot_sdfs(list(out2), title="Super-resolution origianal samples")

In [ ]:
# Optimization 
def volume_estimates(sdfs, dx=1., dy=1., dz=1.): 
    #inside_mask = (-sdfs) > 0
    #volume_estimates = th.sum(inside_mask.float(), dim=list(range(1, sdfs.ndim)))
    volume_estimates_activation = th.sum(StraightThroughEstimator()(-sdfs), dim=list(range(1, sdfs.ndim)))
    #assert th.allclose(volume_estimates, volume_estimates_bin_activation), f"using mask: {volume_estimates}, with activation function {volume_estimates_bin_activation}"
    return volume_estimates_activation * dx * dy * dz

class STEFunction(th.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return (input > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        return F.hardtanh(grad_output)

class StraightThroughEstimator(th.nn.Module):
    def __init__(self):
        super(StraightThroughEstimator, self).__init__()

    def forward(self, x):
        x = STEFunction.apply(x)
        return x
 

def volume_estimates_loss_fn(xs, target_volumes, max_volume=1., grad_var_reg_weight=0, tot_variation_reg_weight=0):
    input_volumes = volume_estimates(xs) / max_volume
    target_volumes = target_volumes / max_volume
    loss = th.nn.MSELoss()(input_volumes, target_volumes)
    if grad_var_reg_weight > 0:
        loss += grad_var_reg_weight * th.var(xs.grad, dim=list(range(1, xs.ndim)))
    if tot_variation_reg_weight > 0:
        loss += tot_variation_reg_weight * tot_variation(xs)
    return loss 

def simple_loss(xs, target_volumes=0., max_volume=1., grad_var_reg_weight=0, tot_variation_reg_weight=0):
    loss = th.nn.MSELoss()(-xs.sum(dim=list(range(1, xs.ndim))), target_volumes)
    return loss

def tot_variation(sdfs, weight=1.):       
    tv_x = ((sdfs[:,:,1:,:,:] - sdfs[:,:,:-1,:,:]).pow(2)).sum()
    tv_y = ((sdfs[:,:,:,1:,:] - sdfs[:,:,:,:-1,:]).pow(2)).sum()    
    tv_z = ((sdfs[:,:,:,:,1:] - sdfs[:,:,:,:,:-1]).pow(2)).sum()
    return tv_x + tv_y + tv_z


In [ ]:
from src.models.utils import identity
def ddim_sample_noise_guidance(
    diffusion,
    denoise_fn,
    x_0,
    from_t_optim_idx,
    obj_fn,
    obj_fn_args={},
    tgt_noise_level = "t_optim",
    clip_denoised=True,
    denoise_kwargs={},
    post_fn=identity,
    #return_intermediates=False,
    log_every_t=5,
    show_pbar=False,
    pbar_kwargs={},
    opt_kwargs={"lr":1e-2, "decay_fn": identity}, 
    grad_clip_value=None,
    plot_debug=False,
):        
    ddim_args = {
        "clip_denoised": clip_denoised,
        "denoise_kwargs": denoise_kwargs, 
        "post_fn": post_fn, 
        #"return_intermediates": return_intermediates, 
        "log_every_t": log_every_t, 
        "show_pbar": show_pbar, 
        "pbar_kwargs": pbar_kwargs
    }

    timesteps = sorted(list(diffusion.use_timesteps))

    # Get the latent at the defined noise level
    x_t = diffusion.ddim_sample_loop(model=denoise_fn, x_t=x_0, inverse=True, to_t_idx=from_t_optim_idx, requires_grad=False)
    if plot_debug:  
        plot_sdfs(x_t, title=f"x_t inverted to timestep {timesteps[from_t_optim_idx]}")

    # Denoise it with guidance
    # if from_t_optim_idx < 0:
    #     idxs = range(from_t_optim_idx, 0)
    # else:
    #     idxs = range(from_t_optim_idx, len(self.ddim_timesteps))
    idxs = range(from_t_optim_idx, 0, -1)
    
    # x_t_optim = x_t.clone()
    # x_t_optim.requires_grad = True
    #optimizer = th.optim.AdamW([x_t_optim], **opt_kwargs)
    with tqdm(idxs, desc="Latent optimization") as pbar_idxs:
        for i, t_idx in enumerate(pbar_idxs):
            #x_t_optim = x_t.clone().detach()
            x_t_optim = x_t.detach()
            x_t_optim.requires_grad_(True)
            if tgt_noise_level == "t_optim":
                tgt_pred = x_t_optim
            elif tgt_noise_level == "zero":
                # with th.enable_grad():
                #     tgt_pred = diffusion.ddim_sample_loop(model=denoise_fn, x_t=x_t_optim, from_t_idx=t_idx)
                tgt_pred = diffusion.ddim_sample_loop(model=denoise_fn, x_t=x_t_optim, from_t_idx=t_idx, requires_grad=True) 
            elif tgt_noise_level == "zero_pred": 
                with th.enable_grad():
                    tgt_pred = diffusion.ddim_sample(model=denoise_fn, x=x_t_optim, t=th.tensor([t_idx] * x_0.shape[0], device=device))["pred_xstart"]
            else:
                raise ValueError("Invalid noise level: available levels are " + ["t_optim", "zero", "zero_pred"] + ".")
            
            # loss_i = obj_fn(tgt_pred, **obj_fn_args)
            # #tgt_pred.requires_grad_(True)
            # optimizer.zero_grad()
            # loss_i.backward()
            # # if grad_clip_value is not None:
            # #     th.nn.utils.clip_grad_value_(x_t, grad_clip_value)
            # optimizer.step()

            
            with th.enable_grad():
                loss_i = obj_fn(tgt_pred, **obj_fn_args) # tgt_pred is a function of x_t
                grad_t = th.autograd.grad(loss_i, x_t_optim, retain_graph=False)[0] # grad of loss wrt to x_t
            print("Grad. norm: ", th.norm(grad_t).item(), " vs x_t norm: ", th.norm(x_t).item())

            # x_{t} optimized 
            decay_fn = opt_kwargs.get("decay_fn", identity)
            x_t = x_t - decay_fn(i) * opt_kwargs["lr"] * grad_t
            x_t.grad = x_t_optim.grad = loss_i.grad = tgt_pred.grad = None

            pbar_idxs.set_postfix({"Loss (mean)": th.mean(loss_i).item()})

            if plot_debug:
                plot_sdfs([tgt_pred, x_t], title=f"Optimization step {i} at timestep {timesteps[t_idx]}", titles=[f"Target shape (target type: \"{tgt_noise_level}\")", f"Optimized shape"]) #(V={th.sum(x_t<0, dim=list(range(1, len(x_t.shape)))).item()})"])

            # x_{t-1} 
            x_t = diffusion.ddim_sample(model=denoise_fn, x=x_t.detach(), t=th.tensor([t_idx] * x_0.shape[0], device=device, requires_grad=False))["sample"]
            # x_t = self.sample_ddim(denoise_fn, x_t=x_t, from_t_idx=-(t_idx), to_t_idx=-(t_idx-1), requires_grad=False, **ddim_args) # x_{t-1}

    return x_t

In [ ]:
shift = -0.061
plot_sdfs(
    [out1, out1 + shift], 
    titles=[
        [f"Original \n $V={volume_estimates(out_i).item()}$" for out_i in out1], 
        [f"Shifting the sdfs by ${shift}$ \n $V={volume_estimates(out_i_shift).item()}$ (incr.: ${((volume_estimates(out_i_shift).item() - volume_estimates(out_i).item())/volume_estimates(out_i).item()):.2f}$)" for out_i_shift, out_i in zip(out1+shift, out1)], 
])

In [ ]:
target_volume_increments = th.tensor([1.1, 1.4, 1.7], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=True,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([1.1, 1.4, 1.7], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=True,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([1.1, 1.4, 1.7], device=device)
t_optim_idx = 25
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
th.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
target_volume_increments = th.tensor([1.1, 1.4, 1.7], device=device)
t_optim_idx = 4
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std[0:1], 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std[0:1]) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=True,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1[0:1], 
        preprocessor1.destandardize(x_edited[0:1]), 
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1[0:1].shape[0])], 
        [f"Edited ($V_{{edit}} = {volume_estimates(x_edited)[i].item():.2f}, V_{{target}}= {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1[0:1].shape[0])],
        ]
)

In [ ]:
shift = -0.062
plot_sdfs(
    [out1, out1 + shift], 
    titles=[
        [f"Original \n $V={volume_estimates(out_i).item()}$" for out_i in out1], 
        [f"Shifting the sdfs by ${shift}$ \n $V={volume_estimates(out_i_shift).item()}$ (incr.: ${((volume_estimates(out_i_shift).item() - volume_estimates(out_i).item())/volume_estimates(out_i).item()):.2f}$)" for out_i_shift, out_i in zip(out1+shift, out1)], 
])

In [ ]:
target_volume_increments = th.tensor([1.3, 1.7, 2.1], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([1.3, 1.7, 2.1], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([1.3, 1.7, 2.1], device=device)
t_optim_idx = 4
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std[0:1], 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std[0:1]) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=True,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1[0:1], 
        preprocessor1.destandardize(x_edited[0:1]), 
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1[0:1].shape[0])], 
        [f"Edited ($V_{{edit}} = {volume_estimates(x_edited)[i].item():.2f}, V_{{target}}= {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1[0:1].shape[0])],
        ]
)

In [ ]:
shift = +0.02
plot_sdfs(
    [out1, out1 + shift], 
    titles=[
        [f"Original \n $V={volume_estimates(out_i).item()}$" for out_i in out1], 
        [f"Shifting the sdfs by ${shift}$ \n $V={volume_estimates(out_i_shift).item()}$ (incr.: ${((volume_estimates(out_i_shift).item() - volume_estimates(out_i).item())/volume_estimates(out_i).item()):.2f}$)" for out_i_shift, out_i in zip(out1+shift, out1)], 
])

In [ ]:
target_volume_increments = th.tensor([-0.3, -0.35, -0.32], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.3, -0.35, -0.32], device=device)
t_optim_idx = 25
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.3, -0.35, -0.32], device=device)
t_optim_idx = 35
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.3, -0.35, -0.32], device=device)
t_optim_idx = 35
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":.5e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.3, -0.35, -0.32], device=device)
t_optim_idx = 20
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
shift = +0.04
plot_sdfs(
    [out1, out1 + shift], 
    titles=[
        [f"Original \n $V={volume_estimates(out_i).item()}$" for out_i in out1], 
        [f"Shifting the sdfs by ${shift}$ \n $V={volume_estimates(out_i_shift).item()}$ (incr.: ${((volume_estimates(out_i_shift).item() - volume_estimates(out_i).item())/volume_estimates(out_i).item()):.2f}$)" for out_i_shift, out_i in zip(out1+shift, out1)], 
])

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 20
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 20
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increments)},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 30
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 10
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 12
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 10
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim",
    opt_kwargs={"lr":1e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the same timestep",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 15
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":.5e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 30
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 20
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.,
        "tot_variation_reg_weight": 1e-1},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 30
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":1e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increments = th.tensor([-0.55, -0.6, -0.65], device=device)
t_optim_idx = 30
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increments),
        "max_volume": 1.,
        "tot_variation_reg_weight": 1e-2},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero_pred",
    opt_kwargs={"lr":.5e-3}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the PREDICTED volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increments[i]):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increment = -0.58
t_optim_idx = 5
x_edited = ddim_sample_noise_guidance(
    ddpm_sampler1,
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={
        "target_volumes": volume_estimates(out1_std) * (1+target_volume_increment),
        "max_volume": 1.},
    from_t_optim_idx=t_optim_idx, 
    tgt_noise_level="zero",
    opt_kwargs={"lr":1e-2}, 
    plot_debug=False,
)

#plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[
        out1, 
        preprocessor1.destandardize(x_edited), 
        #th.abs(out1- preprocessor1.destandardize(x_edited))
    ], 
    title = f"Optimization from the latent at $t={sorted(list(ddpm_sampler1.use_timesteps))[t_optim_idx]}$ based on the volume at the timestep 0",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        #[f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)